In [2]:
import torch
import json
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [5]:
dataset = ImageFolder(root='/Users/arash/Developer/datasets/cub200_toy/images', transform=None)
dataloader = DataLoader(dataset, 32, shuffle=False)

In [6]:
len(dataloader)

32

In [2]:
from PIL import Image
from pathlib import Path
import os

root = Path('/home/akhoeini/scratch/data/miniimagenet/val')

for img in list(root.iterdir())[::-1]:
    if img.name.endswith('JPEG'):
        try:
            img = Image.open(img)
            img.verify()  # This checks if the image is valid
            print("Image is valid and can be opened.")
        except Exception as e:
            print(f"Deleting image: {e}")
            os.system(f'rm {img.absolute()}')

Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid

rm: cannot remove '/home/akhoeini/scratch/data/miniimagenet/val/1429_n02165456.JPEG': No such file or directory


Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid and can be opened.
Image is valid

In [1]:
import torch
import torch.nn.functional as F
class InfoNCELoss(torch.nn.Module):
    def __init__(self, temperature=0.5):
        super(InfoNCELoss, self).__init__()
        self.temperature = temperature

    def forward(self, features):
        """
        Computes the InfoNCE loss.
        
        Args:
            features (torch.Tensor): The feature matrix of shape [2 * batch_size, feature_dim], 
                                     where features[:batch_size] are the representations of 
                                     the first set of augmented images, and features[batch_size:] 
                                     are the representations of the second set.
        
        Returns:
            torch.Tensor: The computed InfoNCE loss.
        """
        # Normalize features to have unit norm
        features = F.normalize(features, dim=1)
        
        # Compute similarity matrix
        similarity_matrix = torch.matmul(features, features.T) / self.temperature

        # Get batch size
        batch_size = features.shape[0] // 2
        
        # Construct labels where each sample's positive pair is in the other view
        labels = torch.arange(batch_size, device=features.device)
        labels = torch.cat([labels + batch_size, labels], dim=0)

        # Mask out self-similarities by setting the diagonal elements to -inf
        mask = torch.eye(2 * batch_size, dtype=torch.bool, device=features.device)
        similarity_matrix = similarity_matrix.masked_fill(mask, -float('inf'))
        
        # InfoNCE loss
        loss = F.cross_entropy(similarity_matrix, labels)
        
        return loss

In [10]:
feats = torch.rand(4,2)
similarity_matrix = torch.matmul(feats, feats.T) / 0.05
labels = torch.arange(2, device=feats.device)
labels = torch.cat([labels + 2, labels], dim=0)
mask = torch.eye(2 * 2, dtype=torch.bool, device=feats.device)
similarity_matrix = similarity_matrix.masked_fill(mask, -float('inf'))

In [11]:
similarity_matrix

tensor([[   -inf, 17.1411, 12.9565,  7.5244],
        [17.1411,    -inf, 11.5164,  7.0868],
        [12.9565, 11.5164,    -inf,  2.4278],
        [ 7.5244,  7.0868,  2.4278,    -inf]])

In [ ]:
labels

tensor([2, 3, 0, 1])

In [13]:
loss = F.cross_entropy(similarity_matrix, labels)

In [14]:
loss

tensor(3.8524)